In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from datetime import datetime

In [2]:
def scrape_data(url):
    results = []

    page = 1
    while True:
        page_url = f"{url}{page}"
        
        headers = {
            "User-Agent": "Tu Agente de Usuario Aquí"
            }
        response = requests.get(page_url, headers=headers)

        
        if response.status_code != 200:
            print(f"Error 200 en página {page}.")
            break

        soup = BeautifulSoup(response.content, 'html.parser')

        items = soup.select("div.listitem")

        if len(items)==0 or items is None:
            print(f"La página {page} no existe o no contiente datos.")
            break

        for item in items:
            date_span = item.find("p", class_="byline").text.strip()
            title = item.find("h3").text.strip()
            summary = item.find("p", class_="teaser").text.strip()

            result = {
                "Fecha": date_span,
                "Titulo": title,
                "Resumen": summary,
                "Fuente": "Zacks"
            }
            results.append(result)

        print(f"Página {page} procesada")
        page += 1
        time.sleep(random.randint(0, 3))

    return results

In [3]:
url_base = "https://www.zacks.com/stock/research/YPF/all-news?page="

resultados = scrape_data(url_base)
df_resultados = pd.DataFrame(resultados)

Página 1 procesada
Página 2 procesada
Página 3 procesada
Página 4 procesada
Página 5 procesada
Página 6 procesada
Página 7 procesada
Página 8 procesada
Página 9 procesada
Página 10 procesada
La página 11 no existe o no contiente datos.


In [4]:
df_resultados_fecha = df_resultados

df_resultados_fecha['Fecha'] = df_resultados['Fecha'].apply(lambda x: x.split("on", 1)[1].strip())

In [5]:
df_resultados_fecha['Titulo'] = df_resultados_fecha.apply(lambda row: row['Titulo'] + ' ' + row['Resumen'], axis=1)
df_resultados_fecha.drop(columns=['Resumen'], inplace=True)

In [6]:
df_resultados_fecha['Fecha'] = df_resultados_fecha['Fecha'].apply(lambda x: datetime.strptime(x.strip(), "%B %d,%Y").strftime("%d/%m/%Y"))

In [9]:
df_resultados_fecha.to_excel('Scraping_Zacks.xlsx',index=False)